***Spectral Indexes Computation***

In [3]:
#Importing
import os
import sys
sys.path.insert(1,'./src')
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import rasterio as rio
from img_list import prs_list

In [4]:
#Parameter Initialization
path_l2d = "../DATABASE/"
name='Gadertal_north_01_prs'
name_img_prs=prs_list[name]
tstart_prs=prs_list[name][0]
tend_prs=prs_list[name][1]
filename=path_l2d+'PRS_L2D_STD_'+tstart_prs+'_'+tend_prs+'_0001'
print(tstart_prs)

20211010102350


In [17]:
#import PRISMA Geotiff (after coregistering it with aerial sensor in QGIS)
#convert Geotiff back to numpy array (easier to manage for reflectance analysis) 
#note that an error will pop up if you don't create the clipped file externally, paste it in the DATABASE folder, 
#and name it in the following way:
#'Clipped_'+name+'_vrf.tif' where name is the parameter in the third box of this notebook
src = rio.open(path_l2d+name+'_cube_coreg.tif')
cube=src.read()
src.close()
print(np.shape(cube))

(230, 1220, 1261)


In [18]:
cube=np.moveaxis(cube,0,2)
vrf_datacube_prs=cube[:,:,:63]
srf_datacube_prs=cube[:,:,63:]
print('vrf shape: ',np.shape(vrf_datacube_prs))
print('srf shape: ',np.shape(srf_datacube_prs))

vrf shape:  (1220, 1261, 63)
srf shape:  (1220, 1261, 167)


In [ ]:
#convert data to reflectance values
reflectance_vrf_prs,reflectance_srf_prs = reflectance_norm(path_l2d,vrf_datacube_prs,srf_datacube_prs,tstart_prs,tend_prs)
print(np.shape(reflectance_vrf_prs))
print(np.shape(reflectance_srf_prs))

In [ ]:
#merge vrf and srf datacubes
reflectance_prs = np.dstack((reflectance_vrf_prs,reflectance_srf_prs))
print('after merge: '+str(np.shape(datacube_prs)))

In [ ]:
#export datacube and wavelength array for backup
if save_files is True:
    np.save(PRISMA_path+'/'+name+'_reflectance',datacube_prs)

In [ ]:
if load_files is True:
    reflectance_prs = np.load(PRISMA_path+'/'+name+'_reflectance.npy')
    wl_prs = np.load(PRISMA_path+'/'+name+'_wl.npy')

In [ ]:
#extracting the index in the vwl1 vector corresponding to R,G,B bands
R_idx_prs=find_nearest(wl_prs,640)
G_idx_prs=find_nearest(wl_prs,560)
B_idx_prs=find_nearest(wl_prs,480)

In [ ]:
#plot histogram of reflectance value to help with histogram stretching
plt.figure(figsize=[20,6])
plt.hist(reflectance_prs.ravel(),density='True',histtype='step',linewidth=2, label='Reflectance histogram') 
plt.legend()
plt.title("Reflectance histogram for "+name)

In [ ]:
rgb = make_rgb_dc(reflectance_prs,wl_prs)
if make_plots is True:
    plt.figure(figsize=(10,10))
    
    
    #uncomment if histogram stretch is needed (enhance image contrast accorsingly to the histogram of the reflectance values)
    threshold=0.7
    rgb[rgb>threshold]=threshold
    rgb=rgb/threshold
    plt.imshow(rgb)